In [1]:
from langchain_community.llms import HuggingFaceTextGenInference
from langchain import PromptTemplate
from typing import NamedTuple
import asyncio

In [2]:
class LlmGenerationParameters(NamedTuple):
    top_k: int
    top_p: float
    temperature: float
    repetition_penalty: float

class RagParameters(NamedTuple):
    chunk_size: int
    chunk_overlap: int
    top_k: int

/home/q36862/#llm_fe/.venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:
template = """
<s>[INST] <<SYS>>
{sys}
<</SYS>>
 
{user} [/INST]
"""
 
prompt = PromptTemplate(
    input_variables=["sys", "user"],
    template=template,
)

In [4]:
def llm_fetch_result(prompt: str, llm_parameter: LlmGenerationParameters) -> str:
    llm = HuggingFaceTextGenInference(
        inference_server_url="http://localhost:15810/",
        max_new_tokens=1024,
        top_k=llm_parameter.top_k,
        top_p=llm_parameter.top_p,
        temperature=llm_parameter.temperature,
        repetition_penalty=llm_parameter.repetition_penalty,
    )
    return llm.stream(prompt)


In [9]:
system_prompt = """
You are an expert programmer and problem-solver, tasked with answering any question provided. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. Do not repeat text. 

Answer the question in Markdown format for readability, use bullet points if possible.
"""
llm = HuggingFaceTextGenInference(
    inference_server_url="http://localhost:15810/",
    max_new_tokens=1024,
    top_k=10,
    top_p=0.7,
    temperature=0.01,
    repetition_penalty=1.03,
    do_sample=True,
)

user_prompt = prompt.partial(sys=system_prompt)
for chunk in llm.stream(user_prompt.format(user="是否可以提供一些臺北市政府的相關資訊。")):
    print(chunk, end="")




當然可以！臺北市政府是臺灣的首都，也是臺灣最大的城市。它位於臺灣的北部，人口超過230萬人。臺北市政府的主要職能包括：

* 公共安全：臺北市政府負責維護臺北市的公共安全，包括警察、消防和救護服務。
* 交通：臺北市政府負責管理臺北市的交通系統，包括公車、捷運和計程車。
* 教育：臺北市政府負責管理臺北市的學校，包括小學、中學和高中。
* 衛生：臺北市政府負責管理臺北市的衛生服務，包括醫院、診所和健康中心。
* 經濟發展：臺北市政府負責推動臺北市的經濟發展，包括創新、創業和就業機會。
* 文化和藝術：臺北市政府負責推動臺北市的文化和藝術活動，包括博物館、劇院和音樂廳。
* 環境保護：臺北市政府負責管理臺北市的環境，包括空氣和水質。
* 社會福利：臺北市政府負責提供臺北市的社會福利服務，包括老人福利、兒童福利和社會保障。

這只是臺北市政府的一些主要職能，但他們的工作範圍非常廣泛，涵蓋了臺北市的各個方面。

In [21]:
system_prompt = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
"""
llm_param = LlmGenerationParameters(top_k=64, top_p=0.9, temperature=0.01, repetition_penalty=1.04)
user_prompt = prompt.partial(sys=system_prompt)
llm_fetch_result(user_prompt.format(user="undertale是什麼樣的遊戲?"), llm_parameter=llm_param)


"\nUndertale是一款2D冒險遊戲，由Toby Fox開發。遊戲設定在一個被稱為地下城的地方，玩家控制的角色是一個名叫薩爾的小狐狸。遊戲的主要特點是其多樣化的故事和角色，以及其中的複雜的道德選擇。\n\n遊戲的故事講述了一個名叫薩爾的小狐狸，他在一個被稱為地下城的地方生活。在這個世界中，所有的生物都是人類，而不是動物。薩爾的目標是通過地下城，並在遊戲中學習如何成為一個更好的人。\n\n遊戲的多樣化的故事和角色使得每次遊戲都能有不同的結果。玩家的選擇會影��hthe game's outcome，從而改變遊戲的故事和角色。\n\nUndertale的多樣化的道德選擇也是其中一個重要的特點。遊戲的道德選擇會影響到遊戲的結局，並且會影響到角色的行為和情緒。這使得遊戲的故事和角色更加真實和複雜。\n\n總的來說，Undertale是一款非常有趣和深入的遊戲，它的多樣化的故事和角色，以及其中的複雜的道德選擇，使得遊戲成為一個令人難忘的體驗。"

In [17]:
template = """
<s>[INST] <<SYS>>
{sys}
<content>
{content}
</content>
<</SYS>>
 
{user} [/INST]
"""


system_prompt_rag="""
You are an expert programmer and problem-solver, tasked with answering any question provided. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. Do not repeat text. 

You should use bullet points in your answer for readability.

Generate a comprehensive and informative answer for the given question based solely on the provided contents.

Anything between the following `context`  html blocks is retrieved from a knowledge bank, not part of the conversation with the user.
"""

content = """
配合本局於110年7月1日組織調整及大數據中心階段性任務完成，為精簡組織俾利提升行政效率，於112年5月24日起停止適用臺北大數據中心作業要點。目前相關業務及人員，皆隸屬於本局數據治理中心，相關研究專案已常態性辦理，其功能仍持續進行中。

本府資訊局為技術幕僚機關，協助各局處有關跨域資料分析，俾供後續局處政策決策參考。凡有局處提出需求，皆盡力配合及行政協處，相關執行專案詳如附件。

臺北市政府資訊局隸屬於臺北市政府，為臺北市政府資訊業務主管機關，主要職掌包括：推動臺北市政管理及便民服務電子化，資訊基礎建設及資通安全管理，以及包括智慧城市、大數據應用等各項數位創新工作的推動。為實踐循證式治理，推動跨域市政資料應用，於110年進行組織調整，正式成立數據治理中心，就資料應用的規範、環境等進行把關。 

臺北大數據中心(Taipei Urban Intelligence Center)為臺北市政府資訊局2020年7月31日成立的跨局處資料應用創新單位，隸屬於資訊局數據治理中心。主責推動跨局處、跨數據源、跨專業的資料應用分析，並且以應用作為驅動力，建構市政跨局處數據整合機制，打造一站式的市政資料流與應用模組，提供市政創新解決方案，邁向循證式決策目標及落實數據治理之理念。  
"""

llm = HuggingFaceTextGenInference(
    inference_server_url="http://localhost:15810/",
    max_new_tokens=1024,
    top_k=64,
    top_p=0.95,
    temperature=0.3,
    repetition_penalty=1.03,
    do_sample=True,
)

prompt = PromptTemplate(
    input_variables=["sys", "user", "content"],
    template=template,
)

user_prompt = prompt.partial(sys=system_prompt)

for chunk in llm.stream(user_prompt.format(user="是否能整理有一些關於台北大數據中心的資訊?", content=content)):
    print(chunk, end="")


為了提升行政效率，臺北市政府資訊局於112年5月24日起停止適用臺北大數據中心作業要點。目前相關業務及人員，皆隸屬於本局數據治理中心，相關研究專案已常態性辦理，其功能仍持續進行中。

臺北大數據中心(Taipei Urban Intelligence Center)為臺北市政府資訊局2020年7月31日成立的跨局處資料應用創新單位，隸屬於資訊局數據治理中心。主責推動跨局處、跨數據源、跨專業的資料應用分析，並且以應用作為驅動力，建構市政跨局處數據整合機制，打造一站式的市政資料流與應用模組，提供市政創新解決方案，邁向循證式決策目標及落實數據治理之理念。

臺北大數據中心的職責包括：

1. 推動臺北市政管理及便民服務電子化，資訊基礎建設及資通安全管理，以及包括智慧城市、大數據應用等各項數位創新工作的推動。
2. 就資料應用的規範、環境等進行把關。
3. 推動跨域市政資料應用，於110年進行組織調整，正式成立數據治理中心，就資料應用的規範、環境等進行把關。
4. 推動跨局處、跨數據源、跨專業的資料應用分析，並且以應用作為驅動力，建構市政跨局處數據整合機制，打造一站式的市政資料流與應用模組，提供市政創新解決方案，邁向循證式決策目標及落實數據治理之理念。
5. 推動跨局處、跨數據源、跨專業的資料應用分析，並且以應用作為驅動力，建構市政跨局處數據整合機制，打造一站式的市政資料流與應用模組，提供市政創新解決方案，邁向循證